#Preparing paths 

In [1]:
import sys

In [2]:
sys.path.append('/opt/opencv-3.0.0/release/lib/python2.7/site-packages/')

In [3]:
#Add the repo to allow import of our classes
sys.path.append('/home/fmpaezri/repos/localization-agent/')

In [4]:
import pickle
import numpy
from PIL import Image
import IPython.display
import os

import tracking.Tester
import tracking.RecurrentTracker
import tracking.TheanoGruRnn
import tracking.VideoSequence as vs
import tracking.VisualAttention
import tracking.GaussianGenerator

Using gpu device 0: Graphics Device (CNMeM is disabled, CuDNN 3007)


#Load evaluation data 

## Random generated or from VOT formatted

In [5]:
#samplesDir = '/tmp/sampleSequences'

#gg = tracking.GaussianGenerator.GaussianGenerator(
#    '/home/fmpaezri/datasets/mscoco/',
#    '/home/fmpaezri/repos/localization-agent/summaries/cocoTrainSummaryCategAndSideGt100SmplsAllCorrected.pkl',
#    ['random', 'sine', 'stretched'], ['stretched'], None, grayscale=False, 
#    objectPathTemplate='./train2014', scenePathTemplate='./train2014'
#        )

#%time gg.saveBatch(4, 0, 60, )
#sequencePath = '/home/datasets/datasets1/vot-challenge/sequences2014/car/'
#sequencePath = os.path.join(samplesDir, numpy.random.choice(os.listdir(samplesDir)))
#%time data, labels = tester.loadImageSequence(sequencePath, newShape=(rnn.imgSize,rnn.imgSize))

###Correction to output ordered points and other preprocessing 

In [6]:
#ONLY FOR VOT BOXES
#Correcting box ccords as expected by IoU
#labelsCopy = labels.copy()
#labelsCopy[:,0] = labels[:,0::2].min(axis=1) 
#labelsCopy[:,1] = labels[:,1::2].min(axis=1)
#labelsCopy[:,2] = labels[:,0::2].max(axis=1) 
#labelsCopy[:,3] = labels[:,1::2].max(axis=1)
#labels = labelsCopy

In [7]:
#data = data[numpy.newaxis, ...].astype(numpy.float32)

In [8]:
#labels = labels[numpy.newaxis,...]

##Load from pickled data 

In [9]:
#Load sequences to evaluate
with open('/data1/vot-challenge/SequencesTestObjectsColor64StillAllScenes.pkl', 'r') as sequenceFile:
    %time sequences = pickle.load(sequenceFile)

CPU times: user 1min 14s, sys: 1min 48s, total: 3min 3s
Wall time: 3min 23s


In [10]:
data, labels = sequences

In [11]:
print data.shape, labels.shape

(64, 60, 224, 224, 3) (64, 60, 4)


#Load model to evaluate 

In [27]:
#I needed to be in the folder with TheanoGruRnn definition
%cd ../tracking

/home/fmpaezri/repos/localization-agent/tracking


In [28]:
#Load the model you want
with open('/data1/vot-challenge/experiments/ECCV_evaluation/expD08/model.pkl', 'r') as modelFile:
    %time rnn = pickle.load(modelFile)

CPU times: user 18 s, sys: 713 ms, total: 18.7 s
Wall time: 19.9 s


In [29]:
tracker = tracking.RecurrentTracker.RecurrentTracker(None, rnn)

In [30]:
tester = tracking.Tester.Tester(tracker)

#Evaluate and visualize 

##Flow calculation 

In [31]:
#reload(tracking.VisualAttention)

In [32]:
#If you find an exception here, something like expected float or something, it might be a change of the function
#of OpenCV for flow calculation between versions, try adding None after the image arguments at the source and reload with the above line
if rnn.computeFlow:
    %time flow = tracking.VisualAttention.computeFlowFromBatch(data)
else:
    flow = None

CPU times: user 3min 5s, sys: 3.01 s, total: 3min 8s
Wall time: 3min 8s


##Evaluation 

In [33]:
%time iou, predLabels = tester.test(data, labels, flow, rnn.batchSize, rnn.imgSize, False, labels.shape[1], 4, '/tmp')

CPU times: user 17.7 s, sys: 4min 47s, total: 5min 5s
Wall time: 5min 5s


In [34]:
iou.mean()

0.46565233658564575

##Visualization 

In [35]:
sample = -1

In [36]:
sample += 1

In [37]:
seq = vs.VideoSequence((Image.fromarray(frame.astype(numpy.uint8)) for frame in data[sample]))

In [38]:
seq.addBoxes(labels[sample], 'green')
seq.addBoxes((predLabels[sample]), 'red')

In [39]:
outputName = '/tmp/testSequence1.mp4'
seq.exportToVideo(30, outputName)

In [40]:
IPython.display.HTML(vs.displayHTML(outputName))